In [2]:
import pennylane as qml

data = qml.data.load("other", name="op-t-mize")


In [28]:
print(type(data)) 
print(len(data))
print(data[0])  
print(data[0].circuit_names)


<class 'list'>
1
<Dataset = attributes: ['circuits', 'circuit_names', ...]>
['mod5_4', 'vbe_adder_3', 'csla_mux_3_original', 'csum_mux_9_corrected', 'qcla_com_7', 'qcla_mod_7', 'qcla_adder_10', 'adder_8', 'rc_adder_6', 'mod_red_21', 'mod_mult_55', 'tof_3', 'tof_4', 'tof_5', 'tof_10', 'gf2^4_mult', 'gf2^5_mult', 'gf2^6_mult', 'gf2^7_mult', 'gf2^8_mult', 'gf2^9_mult', 'gf2^10_mult', 'gf2^16_mult', 'barenco_tof_3', 'barenco_tof_4', 'barenco_tof_5', 'barenco_tof_10', 'gf2^32_mult', 'gf2^64_mult', 'gf2^128_mult', 'gf2^256_mult']


In [3]:
[ds] = qml.data.load("op-t-mize")

In [6]:
pyzx_circuits = []
import pyzx as zx
import tempfile

for i, qscript in enumerate(ds.circuits):
    try:
        # QuantumScript を OpenQASM 2.0 に変換
        qasm_str = qscript.to_openqasm(
            wires=None,             # 自動で wire を割り当て
            rotations=False,        # 観測のための回転ゲートを追加しない
            measure_all=False,      # 全量子ビットを測定しない（PyZXは測定非対応）
            precision=8             # パラメータ精度（省略可）
        )

        # QASM を一時ファイル経由で PyZX に読み込む
        with tempfile.NamedTemporaryFile(suffix=".qasm", mode='w+', delete=False) as tmpfile:
            tmpfile.write(qasm_str)
            tmpfile.flush()
            zx_circuit = zx.Circuit.load(tmpfile.name)

        pyzx_circuits.append(zx_circuit)

    except Exception as e:
        print(f"回路 {i}（{ds.circuit_names[i]}）の変換に失敗: {e}")


In [29]:
import pyzx as zx
import tempfile

pyzx_circuits = []

for i, qscript in enumerate(ds.circuits):
    try:
        # QuantumScript を OpenQASM 2.0 に変換
        qasm_str = qscript.to_openqasm(
            wires=None,
            rotations=False,
            measure_all=False,
            precision=8
        )

        # QASM を一時ファイル経由で PyZX に読み込む
        with tempfile.NamedTemporaryFile(suffix=".qasm", mode='w+', delete=False) as tmpfile:
            tmpfile.write(qasm_str)
            tmpfile.flush()
            zx_circuit = zx.Circuit.load(tmpfile.name)

        # 元の回路名を設定（PyZX Circuit に名前属性を追加）
        zx_circuit.name = ds.circuit_names[i]

        pyzx_circuits.append(zx_circuit)

    except Exception as e:
        print(f"回路 {i}（{ds.circuit_names[i]}）の変換に失敗: {e}")

In [ ]:
print(len(pyzx_circuits))
# PyZX の回路を表示

for i, zx_circuit in enumerate(pyzx_circuits):
    dict = zx_circuit.stats_dict()
    print(dict)
    #print(f"gates:{dict['gates']}, qubits:{dict['qubits']}, depth:{dict['depth']},tcount:{dict['tcount']},had:{dict['had']},ohter:{dict['other']},cnot:{dict['cnot']}")


31
gates:63, qubits:5, depth:0,tcount:28,had:6,ohter:0,cnot:28
gates:150, qubits:10, depth:0,tcount:70,had:10,ohter:0,cnot:70
gates:170, qubits:15, depth:0,tcount:70,had:20,ohter:0,cnot:80
gates:448, qubits:30, depth:0,tcount:196,had:56,ohter:0,cnot:168
gates:443, qubits:24, depth:0,tcount:203,had:39,ohter:0,cnot:186
gates:884, qubits:26, depth:0,tcount:413,had:82,ohter:0,cnot:382
gates:521, qubits:36, depth:0,tcount:238,had:50,ohter:0,cnot:233
gates:900, qubits:24, depth:0,tcount:399,had:80,ohter:0,cnot:409
gates:200, qubits:14, depth:0,tcount:77,had:22,ohter:0,cnot:93
gates:278, qubits:11, depth:0,tcount:119,had:30,ohter:0,cnot:105
gates:119, qubits:9, depth:0,tcount:49,had:14,ohter:0,cnot:48
gates:45, qubits:5, depth:0,tcount:21,had:6,ohter:0,cnot:18
gates:75, qubits:7, depth:0,tcount:35,had:10,ohter:0,cnot:30
gates:105, qubits:9, depth:0,tcount:49,had:14,ohter:0,cnot:42
gates:255, qubits:19, depth:0,tcount:119,had:34,ohter:0,cnot:102
gates:243, qubits:12, depth:0,tcount:112,had:32,

In [27]:
a = pyzx_circuits[0]
print(a)
print(a.to_qasm())

Circuit(5 qubits, 0 bits, 63 gates)
OPENQASM 2.0;
include "qelib1.inc";
qreg q[5];
x q[0];
h q[0];
cx q[1], q[0];
tdg q[0];
cx q[2], q[0];
t q[0];
cx q[1], q[0];
tdg q[0];
t q[1];
cx q[2], q[0];
t q[0];
cx q[2], q[1];
t q[2];
tdg q[1];
cx q[2], q[1];
cx q[1], q[0];
tdg q[0];
cx q[3], q[0];
t q[0];
cx q[1], q[0];
tdg q[0];
t q[1];
cx q[3], q[0];
t q[0];
cx q[3], q[1];
t q[3];
tdg q[1];
h q[0];
cx q[3], q[1];
cx q[1], q[0];
h q[0];
cx q[3], q[0];
tdg q[0];
cx q[4], q[0];
t q[0];
cx q[3], q[0];
tdg q[0];
t q[3];
cx q[4], q[0];
t q[0];
cx q[4], q[3];
t q[4];
tdg q[3];
h q[0];
cx q[4], q[3];
cx q[3], q[0];
h q[0];
cx q[4], q[0];
tdg q[0];
cx q[2], q[0];
t q[0];
cx q[4], q[0];
tdg q[0];
t q[4];
cx q[2], q[0];
t q[0];
cx q[2], q[4];
t q[2];
tdg q[4];
h q[0];
cx q[2], q[4];
cx q[4], q[0];
cx q[2], q[0];

